# Langchain V1 Agents

In [1]:
! pip -q install langchain-community langchain-core langgraph langserve langchain-google-genai wikipedia faiss-cpu langchain-openai google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.8 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from google.colab import userdata
import os

from langchain.agents import tool, AgentExecutor, create_react_agent, Tool, create_self_ask_with_search_agent
from langchain.agents import AgentType, initialize_agent, load_tools, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.tools.retriever import create_retriever_tool
from langchain.utilities import SerpAPIWrapper
from langchain_core.prompts import PromptTemplate

In [3]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
llm = ChatOpenAI()

In [5]:
embedding = OpenAIEmbeddings()

# Tool Calling Agent

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [7]:
search = TavilySearchResults()

In [8]:
search.invoke("What is LangChain?")

[{'url': 'https://python.langchain.com/docs/introduction/',
  'content': "How to use tools in a chain How to migrate from legacy LangChain agents to LangGraph How to use chat models to call tools LangChain is a framework for developing applications powered by large language models (LLMs). Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations. langchain: Chains, agents, and retrieval strategies that make up an application's cognitive architecture. LangServe: Deploy LangChain chains as REST APIs. LangSmith: A developer platform that lets you debug, test, evaluate, and monitor LLM applications. Build stateful, multi-actor applications with LLMs. Integrates smoothly with LangChain, but can be used without it. LangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it."},
 {'url': 'https://www.geeksforgeeks.org/introduction-to-langchain/',
  'content': 'Data Structures and Alg

In [9]:
tools = [search]

In [10]:
prompt = hub.pull("hwchase17/openai-functions-agent")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [12]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
agent_executor.invoke({"input":"tell me about yourself"})



> Entering new AgentExecutor chain...
I am an AI assistant here to help you with any questions or tasks you may have. Feel free to ask me anything, and I'll do my best to assist you!

> Finished chain.


{'input': 'tell me about yourself',
 'output': "I am an AI assistant here to help you with any questions or tasks you may have. Feel free to ask me anything, and I'll do my best to assist you!"}

# RAG Tool

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs = loader.load()

In [18]:
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="\n\n\nLangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024\n\n\n\n\n\nLLMs have paved the way for the dev

In [19]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [20]:
vector = FAISS.from_documents(chunks, embedding)
retriever = vector.as_retriever()

In [21]:
retriever.invoke("What is LangGraph")

[Document(id='465563d0-098a-4b89-993d-925788f28cbd', metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="Since then, we've seen  teams build more complex agentic applications for production; in turn, we've heavily invested in LangGraph, leading to a stable 0.1 release this past June.LangGraph features a persistence layer that enables human-in-the-loop interactions, and it excels at building complex (i.e. more than a single LLM call) applications that require highly domain-specific cognitive architecture. Most of the agents we see in production fit this description.LangGraph is fully open source, available in both Python and Javascript. It works with or without LangChain, and integrate

In [22]:
retriever_tool = create_retriever_tool(retriever,
                                       "langchain",
                                       "search from the langchain documentation")

In [23]:
tools = [search,retriever_tool]

In [24]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [25]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
agent_executor.invoke({"input":"What is LangGraph"})



> Entering new AgentExecutor chain...

Invoking: `langchain` with `{'query': 'LangGraph'}`


LangGraph Studio: The first agent IDE


















































Skip to content
















Case Studies




In the Loop




LangChain




Docs




Changelog





Sign in
Subscribe



















LangGraph Studio: The first agent IDE
LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.

4 min read
Aug 1, 2024

Since then, we've seen  teams build more complex agentic applications for production; in turn, we've heavily invested in LangGraph, leading to a stable 0.1 release this past June.LangGraph features a persistence layer that enables human-in-the-loop interactions, and it excels at building complex (i.e. more than a single LLM call) applications that require highly domain-specific cognitive architecture. Most of the agents we see in production

{'input': 'What is LangGraph',
 'output': 'LangGraph Studio is the first IDE designed specifically for agent development. It provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. LangGraph Studio streamlines the development of agentic applications by offering a new way to develop LLM applications. LangGraph itself is a highly controllable, low-level orchestration framework for building agentic applications. It features a persistence layer that enables human-in-the-loop interactions and excels at building complex applications that require highly domain-specific cognitive architecture. LangGraph is fully open-source, available in both Python and JavaScript, and works with or without LangChain, integrating seamlessly with LangSmith.'}

In [27]:
agent_executor.invoke({"input":"What is weather in new york"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in New York'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1737270567, 'localtime': '2025-01-19 02:09'}, 'current': {'last_updated_epoch': 1737270000, 'last_updated': '2025-01-19 02:00', 'temp_c': 4.4, 'temp_f': 39.9, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 295, 'wind_dir': 'WNW', 'pressure_mb': 1009.0, 'pressure_in': 29.79, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 0, 'feelslike_c': 2.4, 'feelslike_f': 36.2, 'windchill_c': -0.7, 'windchill_f': 30.7, 'heatindex_c': 2.7, 'heatindex_f': 36.9, 'dewpoint_c': 1.5, 'dewpoint_f': 34.7, 'vis_km': 13.0, 'vis_miles'

{'input': 'What is weather in new york',
 'output': 'The current weather in New York is as follows:\n- Temperature: 39.9°F (4.4°C)\n- Condition: Clear\n- Wind: 5.1 mph from WNW\n- Humidity: 83%\n- Visibility: 8.0 miles\n\nFor more detailed forecasts and information, you can visit [Weather API](https://www.weatherapi.com/).'}

# REACT Agent

In [28]:
google_search = SerpAPIWrapper()

In [29]:
tools = [
    Tool(
        name = "Search",
        func=google_search.run,
        description="useful for when you need to answer questions about current events",
        verbose=True
    )
]

In [30]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [31]:
prompt = PromptTemplate.from_template(template)

In [32]:
search_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [33]:
agent_executor = AgentExecutor(agent=search_agent, tools=tools, verbose=True,return_intermediate_steps=True)

In [34]:
agent_executor.invoke({"input": "Where is the hometown of the 2007 US PGA championship winner and his score?"})



> Entering new AgentExecutor chain...
I should search for the hometown of the 2007 US PGA championship winner and his score.
Action: Search
Action Input: "2007 US PGA championship winner hometown and score"['1. United States · Tiger Woods, 71-63-69-69=272 ; 2. United States · Woody Austin, 68-70-69-67=274 ; 3. South Africa · Ernie Els, 72-68-69-66=275 ; T4. United ...', '2007 PGA Championship Final Scores ; Tiger Woods, 71-63-69-69—272 ; Woody Austin, 68-70-69-67—274 ; Ernie Els, 72-68-69-66—275 ; John Senden, 69-70-69-71—279 ; Arron ...', 'When Tiger Woods parred the final hole to claim the 2007 PGA Championship at Southern Hills, it gave him his 13th Major win in just over a ...', "Tiger Woods stretched his lead to three shots after the third round of the 89th US PGA Championship to stand on the verge of capturing the season's final ...", 'The 2007 United States Open Championship was the 107th US Open, held June 14–17 at Oakmont Country Club in Oakmont, Pennsylvania, a suburb north

{'input': 'Where is the hometown of the 2007 US PGA championship winner and his score?',
 'output': 'The 2007 US PGA championship winner was Tiger Woods, with a score of 272, and his hometown is Jupiter, Florida.',
 'intermediate_steps': [(AgentAction(tool='Search', tool_input='2007 US PGA championship winner hometown and score', log='I should search for the hometown of the 2007 US PGA championship winner and his score.\nAction: Search\nAction Input: "2007 US PGA championship winner hometown and score"'),
   '[\'1. United States · Tiger Woods, 71-63-69-69=272 ; 2. United States · Woody Austin, 68-70-69-67=274 ; 3. South Africa · Ernie Els, 72-68-69-66=275 ; T4. United ...\', \'2007 PGA Championship Final Scores ; Tiger Woods, 71-63-69-69—272 ; Woody Austin, 68-70-69-67—274 ; Ernie Els, 72-68-69-66—275 ; John Senden, 69-70-69-71—279 ; Arron ...\', \'When Tiger Woods parred the final hole to claim the 2007 PGA Championship at Southern Hills, it gave him his 13th Major win in just over a 

# ReAct Agent with Custom tools


In [35]:
# Custom tool for the Agent
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}

  return fake_employees.get(name,"Employee not found")# Custom tool for the Agent

In [36]:
# Custom tool for the Agent
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [37]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [38]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [39]:
tools = [get_employee_salary, get_employee_id]

In [40]:
agent = create_react_agent(llm,tools,prompt)

In [41]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [42]:
agent_executor.invoke({"input":"What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
I need to first get the employee ID of Evan.
Action: get_employee_id
Action Input: name = "Evan"Employee not foundIt seems that the employee ID for Evan is not found. Perhaps Evan is not an employee in the database.
Final Answer: Employee not found

> Finished chain.


{'input': 'What is the Salary of Evan?', 'output': 'Employee not found'}

# Self-Ask with Search Agent

In [43]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [44]:
tools = [
    Tool(
        name = "Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to answer questions about current events",
        verbose=True
    )
]

In [45]:
prompt = hub.pull("hwchase17/self-ask-with-search")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [46]:
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

In [47]:
agent = create_self_ask_with_search_agent(llm,tools,prompt)

In [48]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [49]:
agent_executor.invoke(
    {"input":"Which asian country has largest population?"}
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What are the most populous Asian countries?{'type': 'organic_result', 'title': 'Asian Countries by Population (2025) - Worldometer', 'source': 'Worldometer', 'description': 'Asian Countries by population (2025)'}So the final answer is:  This answer is incomplete.  The follow-up question only provides a source to find the answer, not the answer itself.  To complete the answer, one would need to consult the Worldometer link and determine which Asian country has the largest population.

> Finished chain.


{'input': 'Which asian country has largest population?',
 'output': ' This answer is incomplete.  The follow-up question only provides a source to find the answer, not the answer itself.  To complete the answer, one would need to consult the Worldometer link and determine which Asian country has the largest population.'}

In [50]:
agent_executor.invoke(
    {"input":"Who is current captain of indian men's cricket team?"}
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the name of the Indian men's cricket team?the Men in BlueSo the final answer is:  The question cannot be answered with the given information.  The follow-up question did not lead to the answer.  A better follow-up would have been "Who is the current captain of the Indian men's cricket team?"

> Finished chain.


{'input': "Who is current captain of indian men's cricket team?",
 'output': ' The question cannot be answered with the given information.  The follow-up question did not lead to the answer.  A better follow-up would have been "Who is the current captain of the Indian men\'s cricket team?"'}